In [1]:
# %%
import os
from os import path
import pandas as pd
import numpy as np
import glob

# %%
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

# %%
from datasets import load_dataset
from datasets import Dataset, Features, Value


from accelerate import Accelerator
from peft import LoraConfig, get_peft_model

/mnt/home/yucedago/miniconda3/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /mnt/home/yucedago/miniconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /mnt/home/yucedago/miniconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda116.so...


/mnt/home/yucedago/miniconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/mnt/home/yucedago/miniconda3/envs/llm/lib/libcudart.so.11.0'), PosixPath('/mnt/home/yucedago/miniconda3/envs/llm/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


# Definitions

In [2]:
HOME=os.path.expanduser('~')
LIFE2SCENARIO_ROOT_PATH=path.join(HOME,"Documents/life2scenario/")
DATASET_ROOT_PATH=path.join(LIFE2SCENARIO_ROOT_PATH,"life2scenario_minimal/dataset/train/")

print(DATASET_ROOT_PATH)

/mnt/home/yucedago/Documents/life2scenario/life2scenario_minimal/dataset/train/


In [3]:

# %%
PROMPTS_ROOT=path.join(DATASET_ROOT_PATH, "prompts")
REFERENCE_ROOT=path.join(DATASET_ROOT_PATH, "ref_scenarios")
TARGET_ROOT=path.join(DATASET_ROOT_PATH, "target_scenarios")

# %%
PREP_PICKLES_ROOT=path.join(LIFE2SCENARIO_ROOT_PATH, "prep_pickles")

CHECKPOINT="/mnt/home/yucedago/Documents/life2scenario_core/starcoder/bigcode/starcoderbase-1b-merged-210epochs"

# Checkpoint Load

In [4]:
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [5]:
def get_gpus_max_memory(max_memory):
    max_memory = {i: max_memory for i in range(torch.cuda.device_count())}
    return max_memory

orig_checkpoint = "bigcode/starcoderbase-1b"
tokenizer = AutoTokenizer.from_pretrained(orig_checkpoint, load_in_8bit=True)
# to save memory consider using fp16 or bf16 by specifying torch_dtype=torch.float16 for example
model = AutoModelForCausalLM.from_pretrained(CHECKPOINT, torch_dtype=torch.float16,
        use_auth_token=True,
        load_in_8bit=True,
        max_memory=get_gpus_max_memory("70GB"))

/mnt/home/yucedago/miniconda3/envs/llm/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [39]:
model

GPTBigCodeForCausalLM(
  (transformer): GPTBigCodeModel(
    (wte): Embedding(49152, 2048)
    (wpe): Embedding(8192, 2048)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPTBigCodeBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTBigCodeAttention(
          (c_attn): Linear8bitLt(in_features=2048, out_features=2304, bias=True)
          (c_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTBigCodeMLP(
          (c_fc): Linear8bitLt(in_features=2048, out_features=8192, bias=True)
          (c_proj): Linear8bitLt(in_features=8192, out_features=2048, bias=True)
          (act): PytorchGELUTanh()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPTBigCodeBlock(

In [8]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>',
 'additional_special_tokens': ['<|endoftext|>',
  '<fim_prefix>',
  '<fim_middle>',
  '<fim_suffix>',
  '<fim_pad>',
  '<filename>',
  '<gh_stars>',
  '<issue_start>',
  '<issue_comment>',
  '<issue_closed>',
  '<jupyter_start>',
  '<jupyter_text>',
  '<jupyter_code>',
  '<jupyter_output>',
  '<empty_output>',
  '<commit_before>',
  '<commit_msg>',
  '<commit_after>',
  '<reponame>']}

# Load the dataset

In [9]:
train_final = pd.read_csv(path.join(PREP_PICKLES_ROOT, "train_dataset.csv"))

train_final = train_final[["request", "response"]]
train_final.head()

,request,response
0,would you add pedestrian close to hero?\n```\n...,"Here is the result:\n```\n<?xml version=""1.0"" ..."
1,i would like to command you to add pedestrian ...,"Here is the result:\n```\n<?xml version=""1.0"" ..."
2,i would like to request you to add pedestrian ...,"Here is the result:\n```\n<?xml version=""1.0"" ..."
3,i would like to tell you to add pedestrian at ...,"Here is the result:\n```\n<?xml version=""1.0"" ..."
4,add pedestrian close to hero?\n```\n<?xml vers...,"Here is the result:\n```\n<?xml version=""1.0"" ..."


# Generate

In [10]:
def save_model_output(output: str, save_location: str):
    with open(save_location, "w") as f:
        f.write(output)
        print(f"Saved to {save_location}")

In [31]:
def prepare_input_text(example):
    """Prepare the text from a sample of the dataset."""
    text = f"Question: {example}\n\nAnswer: "
    return text


In [32]:
data_prompt = prepare_input_text(train_final["request"][0])
data_prompt

'Question: would you add pedestrian close to hero?\n```\n<?xml version="1.0" ?>\n<OpenSCENARIO>\n  <FileHeader revMajor="1" revMinor="0" date="2020-03-24T12:00:00" description="CARLA:LaneChangeSimple" author=""/>\n  <ParameterDeclarations/>\n  <CatalogLocations/>\n  <RoadNetwork>\n    <LogicFile filepath="Town04"/>\n    <SceneGraphFile filepath=""/>\n  </RoadNetwork>\n  <Entities>\n    <ScenarioObject name="hero">\n      <Vehicle name="vehicle.tesla.model3" vehicleCategory="car">\n        <ParameterDeclarations/>\n        <Performance maxSpeed="69.444" maxAcceleration="200" maxDeceleration="10.0"/>\n        <BoundingBox>\n          <Center x="1.5" y="0.0" z="0.9"/>\n          <Dimensions width="2.1" length="4.5" height="1.8"/>\n        </BoundingBox>\n        <Axles>\n          <FrontAxle maxSteering="0.5" wheelDiameter="0.6" trackWidth="1.8" positionX="3.1" positionZ="0.3"/>\n          <RearAxle maxSteering="0.0" wheelDiameter="0.6" trackWidth="1.8" positionX="0.0" positionZ="0.3"/>\n

In [46]:
inputs = tokenizer.encode(f"{data_prompt}", return_tensors="pt").to(device)
outputs = model.generate(inputs, max_length=30000, max_time=360)
# clean_up_tokenization_spaces=False prevents a tokenizer edge case which can result in spaces being removed around punctuation
response = tokenizer.decode(outputs[0], clean_up_tokenization_spaces=False)

save_model_output(response, "output")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Saved to output


In [47]:
outputs[0][:10]

tensor([9927,   44, 2268,  844, 1015, 3340,  286, 2758,  278, 4230],
       device='cuda:0')

In [48]:
inputs[0][:10]

tensor([9927,   44, 2268,  844, 1015, 3340,  286, 2758,  278, 4230],
       device='cuda:0')

In [49]:
print(response)

Question: would you add pedestrian close to hero?
```
<?xml version="1.0" ?>
<OpenSCENARIO>
  <FileHeader revMajor="1" revMinor="0" date="2020-03-24T12:00:00" description="CARLA:LaneChangeSimple" author=""/>
  <ParameterDeclarations/>
  <CatalogLocations/>
  <RoadNetwork>
    <LogicFile filepath="Town04"/>
    <SceneGraphFile filepath=""/>
  </RoadNetwork>
  <Entities>
    <ScenarioObject name="hero">
      <Vehicle name="vehicle.tesla.model3" vehicleCategory="car">
        <ParameterDeclarations/>
        <Performance maxSpeed="69.444" maxAcceleration="200" maxDeceleration="10.0"/>
        <BoundingBox>
          <Center x="1.5" y="0.0" z="0.9"/>
          <Dimensions width="2.1" length="4.5" height="1.8"/>
        </BoundingBox>
        <Axles>
          <FrontAxle maxSteering="0.5" wheelDiameter="0.6" trackWidth="1.8" positionX="3.1" positionZ="0.3"/>
          <RearAxle maxSteering="0.0" wheelDiameter="0.6" trackWidth="1.8" positionX="0.0" positionZ="0.3"/>
        </Axles>
       

In [45]:
outputs.shape

torch.Size([1, 3985])